## Pre-trained Models

Incredibly, many people from across the world have shared the results of their hard work building and training machine learning models, so that you can take their working models straight off the shelf.

> You can download many model architectures and pre-trained weights from online

One of the largest collections of machine learning models can be found on [HuggingFace](https://huggingface.co/models).

Let's download HuggingFace and some related libraries before we start playing with some of those models.

In [ ]:
!pip install transformers --quiet
!pip install timm --quiet
!pip install pillow --quiet
!pip install requests --quiet

Many of the examples come with implementation code that you can copy directly (always assuming that you understand what it does beforehand, of course).

[Here](https://huggingface.co/SenseTime/deformable-detr) in the documentation, you can find example code that gets the Deformable DETR object detection model from HuggingFace and uses it to make a prediction.

Copying that below allows us to run the model in our own environment:

In [ ]:
from transformers import AutoImageProcessor, DeformableDetrForObjectDetection
import torch
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained("SenseTime/deformable-detr")
model = DeformableDetrForObjectDetection.from_pretrained("SenseTime/deformable-detr")

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.7
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.7)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )


## Be careful of filling up your machine!

> Some downloadable models are huge!

Every time you get a model from HuggingFace, you're losing a big chunk of your hard drive.

You'll want to clear out old models from time to time by deleting them from the command line. I found mine in `~/.cache/torch/hub/` on MacOS, and had to delete them when I encountered a `No space left on device` error. For different operating systems, you might find the location described in this [GitHub issue about deleting models](https://github.com/huggingface/transformers/issues/861).

## With great power, comes great responsibility

Despite how easy it is to copy the code provided online, it is important to understand what all of the code provided in the copyable examples does before copying it blindly.

- Make you understand the input format expected by the model
- Make sure you understand the format of the model output

Without these, it will be hard to integrate your model into your larger application, or will make your implementation prone to error.